<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Import_CV2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 07/12/2021-2
from __future__ import absolute_import

import glob, os, shutil
from os.path import *

try: 
    from google.colab import drive
    from google.colab.patches import cv2_imshow
except Exception as err:
    pass

try: from matplotlib import pyplot as plt
except Exception as err:
    %pip install matplotlib
    from matplotlib import pyplot as plt


try: import cv2
except Exception as err:
    print(err)
    # %pip install opencv-python-headless # works.
    %pip install opencv-contrib-python-headless

import numpy as np

if exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

contentPth = os.getcwd()
tensorflowImagesPath = join(contentPth, 'TensorflowImages')

try:
    drive.flush_and_unmount()
    drive.mount('/content/gdrive/', force_remount=True)
    mydrivePth = '/content/gdrive/MyDrive'
    os.chdir(mydrivePth)
    gdriveTFImagesPth = join(mydrivePth, 'TensorflowImages')
    # os.chdir(gdriveTFImagesPth)
    if not exists(tensorflowImagesPath):
        shutil.copytree(gdriveTFImagesPth,tensorflowImagesPath)

    from TarfileFunctions import *
    from BashColors import C
    os.chdir(contentPth)
except:
    os.chdir(contentPth)
    from TarfileFunctions import *
    from BashColors import C

rootPth = expanduser("~")
tensorflowImagesPath = join(contentPth, 'TensorflowImages')
print(tensorflowImagesPath)

os.chdir(contentPth)
print(f'cv2: {C.IBlue}{cv2.__version__}')
print(f'{os.getcwd()}')

In [ ]:
imagePathList=[]
os.chdir(tensorflowImagesPath)
globImageList=glob.glob('**', recursive=True)
for fil in sorted(globImageList):
    if isfile(fil):
        fullPath=abspath(fil)
        imagePathList.append(fullPath)
        print(fullPath)
print(len(globImageList), 'images')
os.chdir(contentPth)

In [ ]:
#  Create backgroundImg.png
backgroundImg = np.zeros(shape=(56,56,4))
# plt.imshow(backgroundImg)
# print(backgroundImg)
backgroundImgPath = join(tensorflowImagesPath, 'backgroundImage.png')
bgImg = cv2.imwrite(backgroundImgPath, backgroundImg)

cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
bgImg = cv2.imread(backgroundImgPath, cv2.IMREAD_UNCHANGED)
cv2.waitKey(1)
cv2.destroyAllWindows()
zeroPixel = bgImg[0][0]
type(bgImg)


In [ ]:
imagePth = imagePathList[0]
img = cv2.imread(imagePth, cv2.IMREAD_UNCHANGED)
print(type(img))
print('original image:',img.shape)
# cv2_imshow(img)
print()
img = cv2.resize(img, (56,56), cv2.INTER_AREA)
# img[64,64,64] = 0
print('resized image:',img.shape)
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2_imshow(img)
print('bgImg:', bgImg.shape)
cv2_imshow(bgImg)

# zoomImg = cv2.hconcat(backgroundImgPath)


In [ ]:
bgPixel = img[0][0]
type(bgPixel)
count1=0
count2=0
for i in range(img.shape[0]):
    for j in range(img.shape[0]):
        thisPixel = img[i][j]
        if thisPixel[0] == 64:
            thisPixel[0] = 0
        elif thisPixel[1] == 64:
            count1+=1
            thisPixel[1] = 0
        if thisPixel.all() == bgPixel.all():
            count2+=1
            # print(a[3])
        else: pass

        #print(img[i][j])
        
print('count1:', count1)
print('count2:', count2)
print('img.size:', img.size)

In [ ]:

b,g,r,a = cv2.split(img)
t=(b,g,r,a)
print(t)
newImg = cv2.merge([b,g,r,a])
cv2_imshow(newImg)

for px in newImg:
    pass

newImgPath = join(contentPth, 'newEnemy.png')
cv2.imwrite(
    filename=newImgPath,
    img=newImg
)

gdriveNewImgPath = join(mydrivePth, 'newEnemy.png')
cv2.imwrite(
    gdriveNewImgPath,
    newImg
)

cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
print(gdriveTFImagesPth)